### Libraries, paths, and set-up

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from math import sqrt
import os
import pickle
os.chdir('/Users/manotas/Documents/GitHub-Repos/ML-Energy-Colombia')
import warnings
warnings.filterwarnings('ignore')
import pmdarima as pm
from statsmodels.tsa.arima.model import ARIMA
from src.models.metrics import *
from src.utils.utils import datetimer

storing_path = 'data/processed/'

if not os.path.exists('models'):
    os.mkdir('models')

fulldata = pd.read_csv(storing_path + 'fulldata.csv')
fulldata = datetimer(fulldata)

### AR(1) - Baseline
An autogressive approach using only the previous observation: $y_{t} = \phi \cdot y_{t-1} + \epsilon_{t}$
- We define these by grouping plants and their timestamp ranges

In [2]:
AR1_df = fulldata.set_index(['plant', 'datetime'])
AR1_df = np.log(AR1_df[['daily_ask']]).diff().dropna()

In [3]:
# Initializing prediction and actual lists
y_pred_1 = []
y_actual_1 = []

# A dictionary to pickle the models
models_dict = {}

# Fitting the AR(1) model and making predictions
for plant, group in AR1_df.groupby('plant'):
    if len(group) > 1:  # Ensuring we are regressing for plants with more than 1 observation in the timeframe
        model = ARIMA(group['daily_ask'], order=(1, 0, 0))
        model_fit = model.fit(low_memory=False)
        
        # Save each model in the dictionary
        models_dict[plant] = model_fit
        
        # Makes predictions and append to list
        pred = model_fit.predict()
        y_pred_1.append(pred)
        
        # Appends actual values to list
        y_actual_1.append(group['daily_ask'])
        
# Converting lists to pandas series
y_pred_1 = pd.concat(y_pred_1)
y_actual_1 = pd.concat(y_actual_1)

# Ensures predictions and actual values have the same index (proper alignment)
y_pred_1.index = y_actual_1.index

# Pickle the AR(1) models dictionary and save it to a file
# with open('/Users/manotas/Desktop/models/ar1_models.pkl', 'wb') as f:
#    pickle.dump(models_dict, f)

/Users/manotas/miniconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/manotas/miniconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/manotas/miniconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/manotas/miniconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/manotas/miniconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was prov

Using a custom function to return metrics we use across architectures for direct comparison

In [6]:
# Calculate and print performance metrics for training data
print("Naïve model - Test metrics:")
calculate_metrics(y_actual_1, y_pred_1)

Naïve model - Test metrics:
RMSE: 0.21751556000396013
MAE: 0.06192927721710418
sMAPE(0-200): 29.274042622206597%
R-squared: 0.9118171819216949


### ARIMA - Improvement over baseline

In [2]:
ARIMA_df = fulldata.set_index(['plant', 'datetime'])
ARIMA_df = np.log(ARIMA_df[['daily_ask']]).diff().dropna()

In [4]:
# Initialize prediction and actual lists
y_pred_arima = []
y_actual_arima = []

# ARIMA model dictionary
arima_models_dict = {}

# Fitting the ARIMA model and making predictions for each plant
for plant, group in ARIMA_df.groupby('plant'):
    if len(group) > 1:
        # Using auto_arima to find the optimal parameters
        auto = pm.auto_arima(group['daily_ask'], seasonal=False, m=365, start_p=2, max_p=5, 
                             d=1, max_d=1, max_q=3, trace=True, error_action="ignore", 
                             suppress_warnings=True, max_iter=20)
        p, d, q = auto.order

        # Fitting the ARIMA model with the optimal parameters found by auto_arima
        model = ARIMA(group['daily_ask'], order=(p, d, q))
        model_fit = model.fit()

        # Saving each model in the dictionary
        arima_models_dict[plant] = model_fit

        # Making predictions and append to list
        pred = model_fit.predict(start=group.index[0], end=group.index[-1])
        y_pred_arima.append(pred)
        
        # Append actual values to list
        y_actual_arima.append(group['daily_ask'])

# Convert lists to pandas series
predictions_arima = pd.concat(y_pred_arima)
y_actual_arima = pd.concat(y_actual_arima)

# Ensure predictions and actual values have the same index for proper alignment
predictions_arima.index = y_actual_arima.index


Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=-6124.101, Time=20.23 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=24.364, Time=1.48 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-3535.118, Time=0.88 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-4558.847, Time=1.67 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=22.364, Time=0.53 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=-6337.094, Time=15.29 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=-4978.483, Time=4.12 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-5619.655, Time=5.89 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=-6342.460, Time=27.09 sec
 ARIMA(0,1,3)(0,0,0)[0] intercept   : AIC=-5386.561, Time=8.21 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=-6344.497, Time=27.17 sec
 ARIMA(3,1,3)(0,0,0)[0] intercept   : AIC=-6338.529, Time=36.80 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=-6275.795, Time=26.60 sec
 ARIMA(2,1,3)(0,0,0)[0]             : AIC=-6346.541, Time=4.20 sec
 ARIMA(1,1,3)(0,0,0

/Users/manotas/miniconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/manotas/miniconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/manotas/miniconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=1.94 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-6155.171, Time=0.18 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-6205.905, Time=0.39 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-6203.850, Time=0.40 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-6157.163, Time=0.06 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=-6206.160, Time=0.44 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : AIC=-6214.801, Time=0.97 sec
 ARIMA(4,1,0)(0,0,0)[0] intercept   : AIC=-6214.116, Time=0.57 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=-6350.661, Time=1.86 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=-6206.915, Time=1.24 sec
 ARIMA(4,1,1)(0,0,0)[0] intercept   : AIC=-6212.085, Time=0.97 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=-6324.994, Time=1.90 sec
 ARIMA(4,1,2)(0,0,0)[0] intercept   : AIC=-6334.915, Time=2.64 sec
 ARIMA(3,1,1)(0,0,0)[0]             : AIC=inf, Time=0.81 sec

Best model:  ARIMA(3,1,1)(0,0,

/Users/manotas/miniconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/manotas/miniconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/manotas/miniconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=-1512.546, Time=0.89 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-1001.476, Time=0.08 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-1165.897, Time=0.13 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-1236.096, Time=0.26 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-1003.472, Time=0.06 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=-1521.612, Time=1.09 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=-1338.864, Time=0.44 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-1513.008, Time=0.83 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=-1520.200, Time=1.64 sec
 ARIMA(0,1,3)(0,0,0)[0] intercept   : AIC=-1437.371, Time=1.01 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=-1521.368, Time=1.15 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=-1518.129, Time=1.38 sec
 ARIMA(1,1,2)(0,0,0)[0]             : AIC=-1523.496, Time=0.58 sec
 ARIMA(0,1,2)(0,0,0)[0]             : AIC=-1340.849, Time=0.29 sec
 ARIMA(1,1,1)(0,0,0

...

/Users/manotas/miniconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/manotas/miniconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/manotas/miniconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=-211400.516, Time=34.78 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-198569.330, Time=2.79 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-205666.279, Time=3.43 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-208786.874, Time=7.57 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-198571.330, Time=0.87 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=-211836.606, Time=29.74 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=-210092.133, Time=14.30 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-211089.821, Time=14.49 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=-211340.784, Time=46.03 sec
 ARIMA(0,1,3)(0,0,0)[0] intercept   : AIC=-210901.734, Time=17.90 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=-211731.795, Time=23.91 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=-211834.109, Time=50.60 sec
 ARIMA(1,1,2)(0,0,0)[0]             : AIC=-211838.916, Time=4.44 sec
 ARIMA(0,1,2)(0,0,0)[0]             : AIC=-210094.13

/Users/manotas/miniconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/manotas/miniconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/manotas/miniconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=-251370.983, Time=35.31 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-238998.121, Time=2.46 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-244777.969, Time=3.05 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-246717.915, Time=7.97 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-239000.121, Time=1.09 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=-251234.169, Time=29.88 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=-251253.039, Time=27.57 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=-251476.643, Time=37.37 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=-250953.690, Time=34.11 sec
 ARIMA(4,1,2)(0,0,0)[0] intercept   : AIC=-251203.206, Time=58.17 sec
 ARIMA(3,1,3)(0,0,0)[0] intercept   : AIC=-251063.652, Time=61.43 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=-251520.733, Time=49.59 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=-251229.875, Time=47.64 sec
 ARIMA(2,1,3)(0,0,0)[0]             : AIC=-251524.3

/Users/manotas/miniconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/manotas/miniconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/manotas/miniconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/manotas/miniconda3/lib/python3.9/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [8]:
# Pickle the ARIMA models dictionary and save it to a file
with open('/Users/manotas/Documents/GitHub-Repos/ML-Energy-Colombia/models/ARIMAs.pkl', 'wb') as f:
    pickle.dump(arima_models_dict, f)

In [7]:
# Calculate metrics
print('ARIMA(p,d,q) models - Test metrics:')
calculate_metrics(y_actual_arima, predictions_arima)

ARIMA(p,d,q) models - Test metrics:
RMSE: 0.20641988591062507
MAE: 0.05921745814600251
sMAPE(0-200): 19.721823918420156%
R-squared: 0.9205842975863661
